In [47]:
if True:
    # Import modules
    from glob import glob
    from os import makedirs
    import numpy as np
    import matplotlib.pyplot as plt
    from pandas import DataFrame
    from impedance.preprocessing import readCSV
    from galvani.BioLogic import MPRfile
    import ipywidgets
    from IPython.display import display
    from impedance.models.circuits import CustomCircuit
    from impedance.visualization import plot_nyquist, plot_bode, plot_residuals


freq_order = 'descending'

# Convert data from mpr file into csv
filenames = glob('*.mpr')
print('Found {} files'.format(len(filenames)))
for filename in filenames:
    print(filename)
filename = filenames[0]
print('\nFilename chosen: {}'.format(filename))

Found 2 files
220224-PanasonicBR2032-1mAh-C100-no_rest-EIS-10mV_PEIS_C07.mpr
220224-PanasonicBR2032-1mAh-C100-no_rest-EIS-10mV_GCPL_C07.mpr

Filename chosen: 220224-PanasonicBR2032-1mAh-C100-no_rest-EIS-10mV_PEIS_C07.mpr


In [17]:
data = DataFrame(MPRfile(filename).data)
freq = data['freq/Hz']
real_Z = data['Re(Z)/Ohm']
imag_Z = -1*data['-Im(Z)/Ohm']
np.savetxt('data.csv', np.column_stack((freq, real_Z, imag_Z)), delimiter=',')

# Read from csv file and pre-process
freq, Z = readCSV('data.csv')

In [ ]:
# Split the data into individual cycles based on non-monotonous change in frequency
# Find index where adjacent frequencies are ascending/descending
if freq_order == 'ascending':
    indices = np.where(np.diff(freq) < 0)
elif freq_order == 'descending':
    indices = np.where(np.diff(freq) > 0)

makedirs('split', exist_ok=True)

for i in range(len(indices[0])):
    # print('Cycle: {}'.format(i))
    if i == 0:
        start = 0
        end = indices[0][i]
    else:
        start = indices[0][i-1] + 1
        end = indices[0][i]
    # Save the data
    makedirs('split/cycle_{}'.format(i), exist_ok=True)
    np.savetxt('split/cycle_{}/data.csv'.format(i), 
    np.column_stack((freq[start:end], 
    Z[start:end].real, 
    Z[start:end].imag)), 
    delimiter=',')

In [50]:
index = ipywidgets.IntSlider(value=1, description='Step',
min=0, max=199, step=1)
xlim = ipywidgets.IntRangeSlider(value=[0,200], description='xlim', 
min=0, max=1500, step=10)
ylim = ipywidgets.IntRangeSlider(value=[0,200], description='ylim', 
min=0, max=1500, step=10, orientation='horizontal')


def plot_data(index, xlim, ylim):
    plt.ioff()
    fig, ax = plt.subplots(tight_layout=True, facecolor='w')

    freq, Z = readCSV('split/cycle_{}/data.csv'.format(index))
    l = plot_nyquist(ax, Z, fmt='o', color='blue', units='\Omega')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    plt.show()

ipywidgets.interact(plot_data, index=index, xlim=xlim, ylim=ylim, Display=False)


interactive(children=(IntSlider(value=1, description='Step', max=199), IntRangeSlider(value=(0, 200), descript…

<function __main__.plot_data(index, xlim, ylim)>